# Optional: Data Parallelism

https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html

In this tutorial, we will learn how to use multiple GPUs using `DataParallel`.

It’s very easy to use GPUs with PyTorch. You can put the model on a GPU:
```
device = torch.device("cuda:0")
model.to(device)
```

Then, you can copy all your tensors to the GPU:
```
mytensor = my_tensor.to(device)
```

Please note that just calling `my_tensor.to(device)` returns a new copy of `my_tensor` on GPU instead of rewriting `my_tensor`. 
You need to assign it to a new tensor and use that tensor on the GPU.

It’s natural to execute your forward, backward propagations on multiple GPUs. 
However, Pytorch will only use one GPU by default. 

**You can easily run your operations on multiple GPUs by making your model run parallelly using `DataParallel`:**

```
model = nn.DataParallel(model)
```

That’s the core behind this tutorial. We will explore it in more detail below.

## Imports and parameters

##### Import PyTorch modules and define parameters. 

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# Parameters and DataLoaders

input_size = 5
output_size = 2

batch_size = 30
data_size = 100

##### Device 

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
display(device)

device(type='cuda', index=0)

## Dummy DataSet 

##### Make a dummy (random) dataset. You just need to implement the getitem

In [4]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

In [5]:
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size), batch_size=batch_size, shuffle=True)

## Simple Model

For the demo, our model just gets an input, performs a linear operation, and gives an output. 
However, you can use `DataParallel` on any model (CNN, RNN, Capsule Net etc.)

We’ve placed a print statement inside the model to monitor the size of input and output tensors. 
Please pay attention to what is printed at batch rank 0.

In [6]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, _input):
        output = self.fc(_input)
        print("\tIn Model: input size", _input.size(), "output size", output.size())

        return output

## Create Model and DataParallel

This is the core part of the tutorial.

First, we need to make a model instance and check if we have multiple GPUs.

If we have multiple GPUs, we can wrap our model using `nn.DataParallel`. 

Then we can put our model on GPUs by `model.to(device)`

In [7]:
model = Model(input_size, output_size)  # Create instance of Model.

In [8]:
if torch.cuda.device_count() > 1:  # Useful: torch.cuda.device_count()
    
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    
    model = nn.DataParallel(model)  # Key line.

In [10]:
# Turns out don't have more than one GPU here...

In [12]:
model = model.to(device)

## Run the Model

#### Now we can see the sizes of input and output tensors. 

In [15]:
for data in rand_loader:
    _input = data.to(device)
    output = model(_input)
    print("Outside: input size", _input.size(), "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


In [16]:
## Nothing surprising because only one GPU on this machine.

#### **In theory would get:**

**2 GPUs**
```
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
    In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```

**8 GPUs**
```
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```

**Notice the automatic breaking-down and stitching-up that `DataParallel` does for you.

## Summary 

DataParallel splits your data automatically and **sends job orders to multiple models on several GPUs**.

After each model finishes their job, **DataParallel collects and merges the results** before returning it to you.

For more information, please check out https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html.